In [1]:
import glob
import os
import pandas as pd
import re
import numpy as np

def get_10_cv(file_path):
    pattern = "10-CV"
    lines = []
    with open(file_path, 'r') as f:
        # last_line = f.readlines()[-1]
        for line in f:
            if(re.search(pattern,line)):
                if(re.search("final",line)):
                    continue
                else:
                    lines.append(line)
    return lines

def get_result(file_path):
    lines = get_10_cv(file_path)
    try:
        assert len(lines) == 32 or len(lines) == 1, len(lines)
    except:
        print(file_path, f"not done: {len(lines)}")
    if(file_path == ""):
        print(lines)
    cvs, stds = [], []
    for line in lines:
        try:
            time,log_type,msg,cv_txt,std_txt,elapse = line.split('|')
            assert log_type == 'INFO', f"{line}"
            cv = float(cv_txt.split('=')[1])
            std = float(std_txt.split('=')[1])
            cvs.append(cv)
            stds.append(std)
        except Exception as e:
            print(file_path, line, e)
            raise e
    return round(np.array(cvs).mean(),5), round(np.array(stds).mean(),5)

In [7]:
path_files = glob.glob("output/*dependent*-60.log")
df = pd.DataFrame(columns=["subject_setup", "experimental_setup", "preprocess", "segment_lenght", "stimuli", "cv", "std"])
for path_file in path_files:
    # print(f)
    path, filename = os.path.split(path_file)
    # print(path, filename)
    filename, ext = os.path.splitext(filename)
    subject_setup, experimental_setup, stimuli,  preprocess, segment_lenght = filename.split('-')
    # print(subject_setup, experimental_setup, stimuli, preprocess, segment_lenght)
    try:
        cv, std = get_result(path_file)
        row = {
            "subject_setup": subject_setup,
            "experimental_setup": experimental_setup,
            "segment_lenght": segment_lenght,
            "stimuli": stimuli,
            "preprocess": preprocess,
            "cv": cv,
            "std": std
        }
        a = pd.DataFrame.from_records([row])
        df= pd.concat([df, a], axis=0)
    except Exception as e:
            # print(file_path, line, e)
            raise e
    # break
df.to_csv("output/ml.csv")

In [10]:
df = pd.read_csv("output/ml.csv")
for subject_setup in ["dependent","independent"]:
    for experimental_setup in ["segment", 'trial']:
        # for segment_lenght in [1,12,20,60]:
        for segment_lenght in [60]:
        # for segment_lenght in [1]:
            for preprocess in ["DE","DASM","RASM","DCAU","PCC_TIME","PCC_FREQ","PLV","PLI"]:
            # for preprocess in ["PLI"]:
                rows = df.loc[
                    (df["subject_setup"] == subject_setup) &
                    (df["experimental_setup"] ==  experimental_setup) &
                    (df["segment_lenght"] ==  segment_lenght) &
                    (df["preprocess"] ==  preprocess)
                ]
                if(len(rows) == 0): 
                    print(f"{subject_setup}-{experimental_setup}-sti-{preprocess}-{segment_lenght} has not run")
                elif(len(rows) == 1):
                    print(rows)
                    raise ValueError
                else:
                    print(f"{subject_setup}-{experimental_setup}-sti-{preprocess}-{segment_lenght}", round(rows['cv'].mean(),3), round(rows['std'].mean(),3), sep="\t")
                # pass
            # for sti in ["arousal","valence"]:


dependent-segment-sti-DE-60	0.63	0.178
dependent-segment-sti-DASM-60	0.626	0.191
dependent-segment-sti-RASM-60	0.613	0.178
dependent-segment-sti-DCAU-60	0.621	0.182
dependent-segment-sti-PCC_TIME-60	0.627	0.181
dependent-segment-sti-PCC_FREQ-60	0.591	0.155
dependent-segment-sti-PLV-60	0.646	0.176
dependent-segment-sti-PLI-60	0.609	0.154
dependent-trial-sti-DE-60	0.642	0.172
dependent-trial-sti-DASM-60	0.63	0.181
dependent-trial-sti-RASM-60	0.62	0.18
dependent-trial-sti-DCAU-60	0.625	0.173
dependent-trial-sti-PCC_TIME-60	0.636	0.178
dependent-trial-sti-PCC_FREQ-60	0.586	0.154
dependent-trial-sti-PLV-60	0.655	0.166
dependent-trial-sti-PLI-60	0.608	0.155
independent-segment-sti-DE-60	0.409	0.13
independent-segment-sti-DASM-60	0.409	0.116
independent-segment-sti-RASM-60	0.461	0.109
independent-segment-sti-DCAU-60	0.434	0.127
independent-segment-sti-PCC_TIME-60	0.393	0.131
independent-segment-sti-PCC_FREQ-60	0.453	0.12
independent-segment-sti-PLV-60	0.421	0.127
independent-segment-sti-PLI-6

In [4]:
dependent-segment-sti-DE-12	0.691	0.081
dependent-segment-sti-DE-20	0.677	0.108
dependent-segment-sti-DE-60	0.636	0.18
dependent-trial-sti-DE-12	0.63	0.115
dependent-trial-sti-DE-20	0.642	0.123
dependent-trial-sti-DE-60	0.632	0.17
independent-segment-sti-DE-12	0.497	0.076
independent-segment-sti-DE-20	0.453	0.138
independent-segment-sti-DE-60	0.426	0.139
independent-trial-sti-DE-12	0.57	0.028
independent-trial-sti-DE-20	0.628	0.028
independent-trial-sti-DE-60	0.64	0.032

SyntaxError: invalid syntax (384850572.py, line 1)